In [1]:
#!pip install pyswarms

In [2]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))

import pandas as pd
from configparser import ConfigParser
from repo.MyConfigParser import MyConfigParser
from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts, MakeFuture
from repo.FileGenerator import *
from repo.Optimization import SavantPSO

In [3]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/us/test of Juan/config ups us reg.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [5]:
# Get forecasters
bdDf = pd.read_csv(base_path+'New_Cuts_df.csv')#.drop('Unnamed: 0', axis = 1)
bdDf.drop('MAPE', axis=1, inplace=True)
bdDf.drop('period_AE', axis=1, inplace=True)

cuts_df = bdDf.reset_index(drop = True)

cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
cuts_df['periods_for_acc_cal'] = periods
cuts_df['cutoff_month'] = cutoff_month
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [10]:
cuts_df

,market,account_type,funnel,periods_for_acc_cal,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,input_mode,weekday_or_weekend,...,generic_cost_split,competitor_cost_split,new_confirmed,Nov_Dec_2018,Jul_Dec_2019,power,growth,forecaster_name_list,periods,cutoff_month
0,US,Google AdWords,Brand,46.0,0.050,0.01,0.01,multiplicative,multiplicative,additive,...,na,na,na,na,na,na,linear,US Google AdWords Brand.pkl,46.0,2024-02-15
1,US,Google AdWords,Competitor,46.0,0.050,10.00,0.01,additive,multiplicative,na,...,na,na,na,na,na,na,logistic,US Google AdWords Competitor.pkl,46.0,2024-02-15
2,US,Google AdWords,Generic,46.0,0.100,10.00,0.01,multiplicative,multiplicative,na,...,na,na,na,na,na,na,linear,US Google AdWords Generic.pkl,46.0,2024-02-15
3,US,Bing Ads,Brand,46.0,0.100,0.10,0.01,additive,multiplicative,additive,...,na,na,na,na,na,na,logistic,US Bing Ads Brand.pkl,46.0,2024-02-15
4,US,Bing Ads,Competitor,46.0,0.001,0.01,10.00,multiplicative,multiplicative,additive,...,na,na,na,na,na,na,linear,US Bing Ads Competitor.pkl,46.0,2024-02-15
5,US,Bing Ads,Generic,46.0,0.050,0.01,1.00,additive,multiplicative,additive,...,na,na,na,na,na,na,logistic,US Bing Ads Generic.pkl,46.0,2024-02-15


In [9]:
today = datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
fileName
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)
# pos_df = pd.DataFrame({'variable_splits':analytical_file.columns[19:37],'pos': 1})
# pos_df.pos = 1
pos_df

,variable_splits,pos
0,US_Bing_Ads_Brand_Feb_2024,0.941471
1,US_Bing_Ads_Brand_Mar_2024,1.132354
2,US_Bing_Ads_Competitor_Feb_2024,0.907359
3,US_Bing_Ads_Competitor_Mar_2024,1.128617
4,US_Bing_Ads_Generic_Feb_2024,1.060170
5,US_Bing_Ads_Generic_Mar_2024,1.057634
6,US_Google_AdWords_Brand_Feb_2024,1.186637
7,US_Google_AdWords_Brand_Mar_2024,1.145105
8,US_Google_AdWords_Competitor_Feb_2024,0.997527
9,US_Google_AdWords_Competitor_Mar_2024,0.806796


In [11]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: 631591.0
analytical_spend_latest_period: 791749.0
client_budget - analytical_spend_latest_period: 0


In [13]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  1107066.0 difference between planned and forecast X:  0.0
yhat: 627223.0
analytical_spend_latest_period: 791749.0
client_budget - analytical_spend_latest_period: 0


In [16]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

1107065.6700000002 631591.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,US,Bing Ads,Brand,2024-02-15,1680.0,1148.06,Feb,2024,US_Bing Ads_Brand_Feb_2024
1,US,Bing Ads,Brand,2024-02-16,1234.0,982.88,Feb,2024,US_Bing Ads_Brand_Feb_2024
2,US,Bing Ads,Brand,2024-02-17,0.0,340.53,Feb,2024,US_Bing Ads_Brand_Feb_2024
3,US,Bing Ads,Brand,2024-02-18,0.0,300.79,Feb,2024,US_Bing Ads_Brand_Feb_2024
4,US,Bing Ads,Brand,2024-02-19,1611.0,1106.21,Feb,2024,US_Bing Ads_Brand_Feb_2024
...,...,...,...,...,...,...,...,...,...
271,US,Google AdWords,Generic,2024-03-27,398.0,423.15,Mar,2024,US_Google AdWords_Generic_Mar_2024
272,US,Google AdWords,Generic,2024-03-28,392.0,481.16,Mar,2024,US_Google AdWords_Generic_Mar_2024
273,US,Google AdWords,Generic,2024-03-29,345.0,307.96,Mar,2024,US_Google AdWords_Generic_Mar_2024
274,US,Google AdWords,Generic,2024-03-30,0.0,226.25,Mar,2024,US_Google AdWords_Generic_Mar_2024


In [17]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

1107065.7099999997 627223.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,US,Bing Ads,Brand,2024-02-15,1730.0,1390.90,Feb,2024
1,US,Bing Ads,Brand,2024-02-16,1276.0,1190.78,Feb,2024
2,US,Bing Ads,Brand,2024-02-17,0.0,412.56,Feb,2024
3,US,Bing Ads,Brand,2024-02-18,0.0,364.41,Feb,2024
4,US,Bing Ads,Brand,2024-02-19,1659.0,1340.19,Feb,2024
...,...,...,...,...,...,...,...,...
271,US,Google AdWords,Generic,2024-03-27,399.0,547.84,Mar,2024
272,US,Google AdWords,Generic,2024-03-28,394.0,622.95,Mar,2024
273,US,Google AdWords,Generic,2024-03-29,346.0,398.71,Mar,2024
274,US,Google AdWords,Generic,2024-03-30,0.0,292.92,Mar,2024


In [18]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.datetime.strptime(save_date, '_%m_%d_%y').strftime("%Y-%m-%d")
forecast_n_base_df = forecast_n_base_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
forecast_n_base_df = forecast_n_base_df.drop(['Variable_splits'], axis = 1)
# forecast_n_base_df.insert(3, 'WeekStart', [ i for i in forecast_n_base_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
forecast_n_base_df.insert(0, 'Region', region)

forecast_n_base_df[forecast_n_base_df.Ds>='2024-02-01'].sum(numeric_only=1)

OptimalConversionQuantity     631591.00
OptimalSpendValue            1107065.67
BaseConversionQuantity        627223.00
BaseSpend                    1107065.71
dtype: float64

In [19]:
forecast_n_base_df.groupby('Month').sum(numeric_only=1)

,OptimalConversionQuantity,OptimalSpendValue,BaseConversionQuantity,BaseSpend
Month,,,,
Feb,201200.0,315316.67,200366.0,315316.63
Mar,430391.0,791749.00,426857.0,791749.08


# Save to BQ

In [21]:
# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'budget_allocation_scenario_with_base_' + region + save_date + 'jr_test'
table_id = "ups-analytics.forecast_optimization." + target_table_name

# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    forecast_n_base_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 276 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [20]:
forecast_n_base_df[forecast_n_base_df.Ds>='2024-01-01'].to_csv(base_path + 'output.csv')